In [109]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix

In [110]:
lines = []
lines_test = []

categories = [1, 2, 3, 4, 5]
with open("train.dat", "r") as fh:
    lines = fh.readlines()
    
lines_train = []
classes = []
for s in lines:
    temp = s.split("\t")
    classes.append(temp[0])
    lines_train.append(temp[1])
    
with open("test.dat", "r") as fh:
    lines_test = fh.readlines()

In [111]:
print(len(lines_test))
# print(classes)
print(len(lines_train))

14442
14438


In [112]:
# train1 = [l.split() for l in lines_train]
# test1 = [l.split() for l in lines_test]


def filterLen(docs, minlen):
    r""" filter out terms that are too short. 
    docs is a list of lists, each inner list is a document represented as a list of words
    minlen is the minimum length of the word to keep
    """
#     return [ [t for t in d if len(t) >= minlen ] for d in docs ]
    temp = ""
    for d in docs:
#         d = d.sub('[^A-Za-z0-9]+', '', string)
#         d = ''.join(e for e in d if e.isalnum())
#         temp+=d+" "
        if len(d)>minlen:
            temp+=d+" "
    return temp;
#     return [d for d in docs if len(d)>minlen]



def removeUnwantedCharacters(data):
    docs_temp = []
    for l in data:
        temp = l.split()
        docs_temp.append(filterLen(temp, 4))
    return docs_temp

docs_train = removeUnwantedCharacters(lines_train)
docs_test = removeUnwantedCharacters(lines_test)

print(type(docs_train))

# docs_train = filterLen(train1, 4)
# docs_test = filterLen(test1, 4)
# print(docs_train[:5][:5])

<type 'list'>


In [113]:
print(type(docs_train))

print(type(docs_train[0]))

<type 'list'>
<type 'str'>


In [114]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

from time import time

In [115]:
vectorizer = HashingVectorizer(stop_words='english')
X_train = vectorizer.transform(docs_train)

# vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')
# X_train = vectorizer.fit_transform(docs_train)
print(X_train.shape)

y_train = np.asarray(classes)
print(y_train)
print(len(y_train))

(14438, 1048576)
['4' '5' '2' ..., '1' '2' '3']
14438


In [116]:
X_test = vectorizer.transform(docs_test)
# print(X_test)

In [117]:
# target_names = ["1", "2", "3", "4", "5"]
def classify(clf):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time()
    pred = clf.predict(X_test)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)
    print(type(pred))
    return pred

In [118]:
pred1 = classify(KNeighborsClassifier(n_neighbors=(int(len(lines_train)**0.5))))


________________________________________________________________________________
Training: 
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=120, p=2,
           weights='uniform')
train time: 0.007s
test time:  15.781s
<type 'numpy.ndarray'>


In [119]:
print(pred1[:10])

['5' '5' '5' '2' '1' '4' '5' '5' '1' '5']


In [120]:
pred2 = classify(KNeighborsClassifier(n_neighbors=115))

________________________________________________________________________________
Training: 
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=115, p=2,
           weights='uniform')
train time: 0.009s
test time:  14.664s
<type 'numpy.ndarray'>


In [121]:
print(pred2[:10])

['5' '5' '5' '2' '1' '4' '1' '5' '1' '5']


In [122]:
from sklearn.linear_model import SGDClassifier
# pred3 = SGDClassifier(alpha=.0001, n_iter=50, penalty=penalty)
pred3 = classify(SGDClassifier(alpha=.0001, n_iter=50))


________________________________________________________________________________
Training: 
SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=50,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)


/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


train time: 2.302s
test time:  0.048s
<type 'numpy.ndarray'>


In [123]:
print(pred3[:10])

['5' '5' '5' '5' '1' '3' '5' '5' '1' '4']


In [124]:
from sklearn.linear_model import PassiveAggressiveClassifier
# pred4 = classify(PassiveAggressiveClassifier(n_iter=50))
pred4 = classify(PassiveAggressiveClassifier())


________________________________________________________________________________
Training: 
PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
              fit_intercept=True, loss='hinge', max_iter=None, n_iter=None,
              n_jobs=1, random_state=None, shuffle=True, tol=None,
              verbose=0, warm_start=False)
train time: 0.270s
test time:  0.033s
<type 'numpy.ndarray'>


In [125]:
print(pred4[:30])

['1' '5' '1' '5' '1' '3' '5' '5' '1' '3' '3' '4' '4' '2' '2' '1' '3' '1'
 '4' '4' '3' '1' '5' '4' '1' '1' '4' '4' '1' '3']


In [126]:
def findMatchPercentage(p1, p2):
#     p1 = p1.tolist()
#     p2 = p2.tolist()
    match = 0
    total = len(p1)
    for i in range(0,total-1):                
        if(p1[i]==p2[i]):
            match = match+1
    per = ((float(match)/float(total))*100)
    return per

In [127]:
print(findMatchPercentage(pred1, pred2))

98.0473618612


In [128]:
def writeToFile(pred):
    with open('output3.dat','w+') as f:
        for p in pred:
            f.write(str(p)+"\n")

In [16]:
writeToFile(pred1)

In [136]:
def readFile(filename):
    content = []
    with open(filename) as f:
        content = f.readlines()
        content = [x.strip() for x in content] 
    return content
# you may also want to remove whitespace characters like `\n` at the end of each line


In [147]:
p1 = readFile("output4.dat")
print(findMatchPercentage(p1, pred1.tolist()))

90.8530674422
